# Data preparation

I converted the JSON dataset into a relational database format to facilitate detailed analysis. Utilizing Django, I stored the data in a PostgreSQL database. The following code snippets outline the process of constructing the database with various tables.

In [38]:
import json

In [3]:
with open('data.json') as f:
    data = json.load(f)

In [ ]:
# List of Resstaurants where no sublocations are present

location_without_sublocations = []
restaurants_lws = 0
for sub in data:
    try:
        location_without_sublocations.append(sub)
        res = data[sub]['restaurants']
        restaurants_lws +=1              
        mainloc = MainLocation(name = sub)
        # subloc= SubLocation(name = sub)
        mainloc.save()
        # subloc.save()
        loc = Location(main = mainloc)
        loc.save()               
        for rest in data[sub]['restaurants']:
            try:
                restaurant = Restaurant(
                    rid = rest,
                    name = data[sub]['restaurants'][rest]['name'],
                    rating = data[sub]['restaurants'][rest]['rating'],
                    rating_count = data[sub]['restaurants'][rest]['rating_count'],
                    cost = data[sub]['restaurants'][rest]['cost'],
                    address = data[sub]['restaurants'][rest]['address'],
                    cuisine = data[sub]['restaurants'][rest]['cuisine'],
                    lic_no = data[sub]['restaurants'][rest]['lic_no'],
                    location = loc,
                    link =  data[sub]['restaurants'][rest]['link']
                )
                restaurant.save()
            except:
                try:
                    new = data[sub]['restaurants'][rest]['price']
                    restaurant = Restaurant(
                        rid = rest,
                        name = data[sub]['restaurants'][rest]['name'],
                        rating = data[sub]['restaurants'][rest]['rating'],
                        cost = data[sub]['restaurants'][rest]['price'],
                        cuisine = data[sub]['restaurants'][rest]['type'],
                        link =  data[sub]['restaurants'][rest]['link']
                    )
                    restaurant.save()
                except:
                    print(data[sub]['restaurants'][rest]['price'])
                    pass
    except:
        pass
        # print(data[sub].keys())

In [ ]:

# List of Resstaurants in cities where sublocations are present

sublocations = []
location_with_sublocations = []
restaurants_in_sublocations = 0
for sub in data:
    try:
        res = data[sub]['restaurants']
        # location_with_sublocations.append(sub)
        # restaurants_in_sublocations +=1
    except:
        for s in data[sub]:
            if s != 'link':
                # print(s)
                try:
                    sublocations.append(s)
                    res = data[sub][s]['restaurants']
                    restaurants_in_sublocations +=1                 
                    mainloc = MainLocation(name = sub)
                    subloc= SubLocation(name = s)
                    mainloc.save()
                    subloc.save()
                    loc = Location(main = mainloc, sub = subloc)
                    loc.save()               
                    for rest in data[sub][s]['restaurants']:
                        try:
                            restaurant = Restaurant(
                                        rid = rest,
                                        name = data[sub][s]['restaurants'][rest]['name'],
                                        rating = data[sub][s]['restaurants'][rest]['rating'],
                                        rating_count = data[sub][s]['restaurants'][rest]['rating_count'],
                                        cost = data[sub][s]['restaurants'][rest]['cost'],
                                        address = data[sub][s]['restaurants'][rest]['address'],
                                        cuisine = data[sub][s]['restaurants'][rest]['cuisine'],
                                        lic_no = data[sub][s]['restaurants'][rest]['lic_no'],
                                        location = loc,
                                        link =  data[sub][s]['restaurants'][rest]['link'])
                            restaurant.save()
                        except:
                            try:
                                new = data[sub][s]['restaurants'][rest]['price']
                                restaurant = Restaurant(
                                    rid = rest,
                                    name = data[sub][s]['restaurants'][rest]['name'],
                                    rating = data[sub][s]['restaurants'][rest]['rating'],
                                    cost = data[sub][s]['restaurants'][rest]['price'],
                                    cuisine = data[sub][s]['restaurants'][rest]['type'],
                                    link =  data[sub][s]['restaurants'][rest]['link']
                                )
                                restaurant.save()
                            except:
                                print(data[sub][s]['restaurants'][rest]['price'])
                                pass
                except:
                    print(data[sub][s])

# Data retriving, cleaning and saving from the database

Data was retrived from the database, cleand and saved in the csv format 

In [1]:
import pandas as pd

In [ ]:
# all_restaurants = Restaurant.objects.all()
all_restaurants = Restaurant.objects.values('id', 'location__main__name','location__sub__name', 'name', 'rating', 'rating_count', 'cost', ).annotate(total_restaurants=Count('id'))
restaurant_df = pd.DataFrame(all_restaurants)
restaurant_df['costs'] = restaurant_df['cost'].str.extract(r'(\d+)')
restaurant_df['costs'] = pd.to_numeric(restaurant_df['costs'])
# restaurant_df['rating_counts'] = pd.to_numeric(df['rating_count'].str.replace(r'\D+', ''), errors='coerce').fillna(0).astype(int)
restaurant_df['rating_counts'] = df['rating_count'].str.extract(r'(\d+)').fillna(0).astype(int)
restaurant_df['ratings'] = pd.to_numeric(df['rating'].str.replace(r'\D+', ''), errors='coerce').fillna(0).astype(int)
# restaurant_df_new = restaurant_df.drop(columns=['rid', 'cost', 'address', 'lic_no', 'link', 'rating_count'])
restaurant_df_new = restaurant_df.drop(columns=['cost', 'rating_count', 'rating'])

In [ ]:
restaurant_df_new.to_csv('clean_swiggy_data.csv', index=False)


In [ ]:
restaurants_df = pd.read_csv('clean_swiggy_data.csv')

# Some primary results were generated during the ETL process

In [43]:
# Number of Cities without subregions
len(set(data.keys()))

623

In [44]:
# Number of Cities have subregions
len(set(location_with_sublocations))

26

In [45]:
# Total number of subregions
len(set(sublocations))

373

In [46]:
# Duplicate names of subregion 
duplicate_sublocations = [x for x in sublocations if sublocations.count(x) > 1]
duplicate_sublocations

['Malviya Nagar', 'Pratap Nagar', 'Malviya Nagar', 'Pratap Nagar']

The set of subregions was less then the original list because there were duplicates

# 1.  How many cities (including subregions) where Swiggy is having its restaurants listed

Total number of cities including the subregions is **996**, **26** cities have subrerions and total subregion is **373**.

In [22]:
totallocation  = len(data.keys()) + len(set(sublocations))
totallocation

996

# 2. How many cities  (don't include subregions) where Swiggy is having their restaurants listed

Total number of main Cities is **623**

In [50]:
len(set(data.keys()))

623

# 3 The Subregion of Delhi with the maximum number of restaurants listed on Swiggy

In Delhi, Indirapuram have the max number of **1279** restaurants.

In [208]:
grouped_df_delhi = restaurants_df[restaurants_df['location__main__name'] == 'Delhi'].groupby(['location__sub__name']).agg(
    number_of_restaurants=('name', 'count')
)
sorted_df_delhi = grouped_df_delhi.sort_values(by='number_of_restaurants', ascending=False)

sorted_df_delhi.head(10)

,number_of_restaurants
location__sub__name,
Indirapuram,1279
Rohini,1137
Greater Kailash 2,1038
Laxmi Nagar,933
Malviya Nagar,900
Dwarka,721
Rajouri Garden,710
GTB Nagar,685
Dilshad Gardens,677


# 4. Name the top 5 Most Expensive Cities in the Datasets.

I have used meidan insted of mean value. Based on the result I could not determine the top 5, so I made a table of top 11. 


In [213]:
grouped_df_mec = restaurants_df.groupby(['location__main__name']).agg(
    Costs=('costs', 'median')
)

sorted_df_mec = grouped_df_mec.sort_values(by='Costs', ascending=False)
sorted_df_mec.head(11)

,Costs
location__main__name,
Gangtok,500.0
Mussoorie,450.0
Diu,400.0
Darjeeling,400.0
North-goa,400.0
Rangpo,400.0
Budhwal,400.0
Central-goa,400.0
Kohima,400.0


# 5. List out the top 5 Restaurants with Maximum & minimum ratings throughout the dataset


I have used mean for rating and sum for the total number of ratings count. The avg rating may not reresent the correct evaluation, as, a restaurant may have many outlates and number of rating count also could be heigher than the a restauent have 1 outlate with few visitor. From the below tables we can see that most popular restaurants with many outlates have very poor avg ratings.

In [232]:
grouped_dataset = restaurants_df.groupby(['name']).agg(costs =('costs', 'mean'), avg_ratings = ('ratings', 'mean'), num_ratings = ('rating_counts','sum'), outlates = ('name','count'))

In [239]:
grouped_dataset.sort_values(by='avg_ratings', ascending=False).head(5)


,costs,avg_ratings,num_ratings,outlates
name,,,,
Sahu juice,300.0,5.0,20,1
VASAVI UPAHARA,150.0,5.0,20,1
BunnyBee Cafe,200.0,5.0,0,1
POURTUA- FOR U,200.0,5.0,0,1
RS food point,149.0,5.0,50,1


In [238]:
grouped_dataset.sort_values(by='avg_ratings', ascending=False).tail(5)

,costs,avg_ratings,num_ratings,outlates
name,,,,
KARTHIK COOL AND CHAT CORNER,40.0,0.0,0,1
KARTHIK CHATS,150.0,0.0,0,1
KARTARPURA JUNCTION,200.0,0.0,0,1
KARPAGAM RESTAURANT,200.0,0.0,0,1
zzz pizza,400.0,0.0,0,1


In [240]:
grouped_dataset.sort_values(by='num_ratings', ascending=False).head(5)


,costs,avg_ratings,num_ratings,outlates
name,,,,
Pizza Hut,359.852941,1.523529,17828,340
Domino's Pizza,405.260304,1.336226,15445,461
Kwality Walls Frozen Dessert and Ice Cream Shop,251.302548,1.445860,11923,314
KFC,409.905488,1.588415,11839,328
Faasos - Wraps & Rolls,222.018349,1.830275,10654,218


In [241]:
grouped_dataset.sort_values(by='num_ratings', ascending=False).tail(5)

,costs,avg_ratings,num_ratings,outlates
name,,,,
K K S TAKE AWAY,200.0,0.0,0,1
K K S Bakers,150.0,0.0,0,1
K K Kebab Corner & Restaurant,275.0,0.0,0,1
K K KONAR HOTEL CATERING,300.0,0.0,0,1
zzz pizza,400.0,0.0,0,1


# 6. Name of top 5 cities with the highest number of restaurants listed.


In [215]:
# Name of top 10 cities with the highest number of restaurants listed.

grouped_df_total = restaurants_df.groupby(['location__main__name']).agg(
    number_of_restaurants=('name', 'count')
)

sorted_df_total = grouped_df_total.sort_values(by='number_of_restaurants', ascending=False)
sorted_df_total.head(10)


,number_of_restaurants
location__main__name,
Bangalore,15302
Delhi,14080
Pune,12438
Hyderabad,10011
Chennai,9954
Kolkata,8282
Mumbai,6099
Gurgaon,3599
Ahmedabad,3541


# 7. Top 10 cities as per the number of restaurants listed

I have listed the top 10 cities in Q 6. So, here I have listed the top subregions with total restautant number

In [216]:
# Top 10 cities as per the number of restaurants listed

grouped_df_top = restaurants_df.groupby(['location__main__name','location__sub__name'])['total_restaurants'].sum().reset_index()

# Sort in descending order
grouped_df_top = grouped_df_top.sort_values(by='total_restaurants', ascending=False)

grouped_df_top.head(10)

,location__main__name,location__sub__name,total_restaurants
113,Delhi,Indirapuram,1279
13,Bangalore,BTM,1172
126,Delhi,Rohini,1138
265,Pune,Kothrud,1089
23,Bangalore,Indiranagar,1086
140,Gurgaon,sohna road,1081
19,Bangalore,Electronic City,1049
110,Delhi,Greater Kailash 2,1038
238,Mumbai,Vashi,1022
158,Hyderabad,Kukatpally,1009


# 8. Name the top 5 Most Popular Restaurants in Pune

In [218]:
# Name the top 5 Most Popular Restaurants in Pune
grouped_df_pune = restaurants_df[restaurants_df['location__main__name'] == 'Pune'].groupby(['name'])\
    .agg({'costs': 'mean', 'ratings': 'mean', 'rating_counts':'sum', 'name':'count'})

In [219]:
# Name the top 5 Most Popular Restaurants in Pune based on number of ratings
grouped_df_pune.sort_values(by='rating_counts', ascending=False).head(5)

,costs,ratings,rating_counts,name
name,,,,
NH1 Bowls - Highway To North,250.000000,1.272727,1290,22
Mother's Kitchen,266.666667,2.333333,1120,6
Kwality Walls Frozen Dessert and Ice Cream Shop,244.545455,1.909091,1060,22
NAADBRAMHA IDLI,172.222222,0.888889,1000,9
Subway,347.368421,1.736842,910,19


In [220]:
# Name the top 5 Most Popular Restaurants in Pune based on avg ratings

grouped_df_pune.sort_values(by='ratings', ascending=False).head(5)

,costs,ratings,rating_counts,name
name,,,,
The Iceberg Cafe,150.0,5.0,20,1
Mannat,300.0,5.0,20,1
La Sicilia Bistro & Patisserie,300.0,5.0,20,1
Cake Glory,500.0,5.0,20,1
NOTO Healthy Ice Cream,250.0,5.0,20,1


# 9. Which SubRegion in Delhi is having the least expensive restaurant in terms of cost

In [223]:
# SubRegions in Delhi having the most and least expensive restaurant in terms of cost

grouped_df_delhi2 = restaurants_df[restaurants_df['location__main__name'] == 'Delhi'].groupby(['location__main__name', 'location__sub__name']).agg(
    {'costs': 'mean', 'ratings': 'mean', 'rating_counts':'mean'}
)
sorted_df_delhi2 = grouped_df_delhi2.sort_values(by='costs', ascending=False)

# SubRegions in Delhi having the most expensive restaurant
print (sorted_df_delhi2.head(5))

# SubRegions in Delhi having the least expensive restaurant
print (sorted_df_delhi2.tail(5))


                                               costs   ratings  rating_counts
location__main__name location__sub__name                                     
Delhi                BBK_Chattarpur       800.000000  0.000000       0.000000
                     Khan Market          600.915888  0.831776      20.392523
                     Vasant Kunj          439.375000  1.361842      38.521930
                     Connaught Place      434.449612  0.728682      14.740310
                     Greater Kailash New  406.172840  1.407407      46.000000
                                               costs   ratings  rating_counts
location__main__name location__sub__name                                     
Delhi                Rohini               277.913808  1.246924      30.238137
                     Mayur Vihar          275.059259  1.168889      26.739259
                     Laxmi Nagar          270.195070  0.951768      25.544480
                     Uttam Nagar          253.321970  1.636364  

# 10. Top 5 most popular restaurant chains in India

In [224]:
# Top 5 most popular restaurant chains in India based on rating count

grouped_df_all = restaurants_df.groupby('name')['rating_counts'].sum().reset_index()

# Sort the grouped DataFrame by 'total_restaurants' in descending order
grouped_df_all = grouped_df_all.sort_values(by='rating_counts', ascending=False)

# sorted_df.head(5)
grouped_df_all.set_index('name').head(5)

,rating_counts
name,
Pizza Hut,17828
Domino's Pizza,15445
Kwality Walls Frozen Dessert and Ice Cream Shop,11923
KFC,11839
Faasos - Wraps & Rolls,10654


# 11.  Which restaurant in Pune has the most number of people visiting

**NH1 Bowls - Highway To North** is the restaurant where most number of people are visiting at Pune.

In [225]:
# Which restaurant in Pune has the most number of people visiting based on number of ratings.
grouped_df_pune.sort_values(by='rating_counts', ascending=False).head(5)

,costs,ratings,rating_counts,name
name,,,,
NH1 Bowls - Highway To North,250.000000,1.272727,1290,22
Mother's Kitchen,266.666667,2.333333,1120,6
Kwality Walls Frozen Dessert and Ice Cream Shop,244.545455,1.909091,1060,22
NAADBRAMHA IDLI,172.222222,0.888889,1000,9
Subway,347.368421,1.736842,910,19


# 12. Top 10 Restaurants with Maximum Ratings in Banglore

I have creatd two table, one based on ave ratings and another number of ratings.

In [227]:
# Top 10 Restaurants with Maximum Ratings in Banglore

grouped_df_banglore = restaurants_df[restaurants_df['location__main__name'] == 'Bangalore'].groupby(['name']).agg({'costs': 'mean', 'ratings': 'mean', 'rating_counts':'mean', 'name':'count'})

sorted_df_ratings_banglore = grouped_df_banglore.sort_values(by='ratings', ascending=False)
sorted_df_rating_counts_banglore = grouped_df_banglore.sort_values(by='rating_counts', ascending=False)

print(sorted_df_ratings_banglore.head(10))

print(sorted_df_rating_counts_banglore.head(10))

                              costs  ratings  rating_counts  name
name                                                             
SLV Food Joint                150.0      5.0           20.0     1
Cattle cream                  250.0      5.0           20.0     1
ROLLING DOUGH CAFE            700.0      5.0           20.0     1
Dessert Crunch                300.0      5.0           20.0     1
Sri Manjunatha refreshments   150.0      5.0           20.0     1
Poha House                    120.0      5.0           20.0     1
Om Ganesh fruit juice Center  100.0      5.0           20.0     1
meena food zone               300.0      5.0           20.0     1
Aahar Vihar                   250.0      5.0           20.0     1
Appu Biriyani Adda            250.0      5.0           20.0     1
                           costs  ratings  rating_counts  name
name                                                          
The Bake Feast             200.0      4.0          500.0     1
Red Earth Coffee Ro

# 13. Top 10 Restaurant in Patna w.r.t rating

I could not find top 10 based on rating or rating_count, because 10 restaurant have same avg ratings and rating_count.

In [228]:
grouped_df_patna = restaurants_df[restaurants_df['location__main__name'] == 'Patna'].groupby(['name'])\
    .agg(costs =('costs', 'mean'), avg_ratings = ('ratings', 'mean'), num_ratings = ('rating_counts','sum'), outlates = ('name','count'))


In [229]:
grouped_df_patna.sort_values(by='avg_ratings', ascending=False).head(50)

,costs,avg_ratings,num_ratings,outlates
name,,,,
FRIENDS CAFE,200.0,4.0,50,1
FOOD TALKIES,400.0,4.0,50,1
THE BACKSTREET KITCHEN,200.0,4.0,20,1
Dragon Flames Delivery,400.0,4.0,100,1
Dragon Xpress,300.0,4.0,20,1
Drink Shakes,100.0,4.0,100,1
THE ARTISAN HOUSE BAKERY AND CAFE,500.0,4.0,500,1
THALI,250.0,4.0,100,1
Fauji Da Dhaba,150.0,4.0,50,1


In [230]:
grouped_df_patna.sort_values(by='num_ratings', ascending=False).head(50)

,costs,avg_ratings,num_ratings,outlates
name,,,,
SURBHI DHABA,400.0,3.0,500,1
Moti Mahal Delux,300.0,3.0,500,1
MASALA DARBAR,200.0,3.0,500,1
Rasoi Ghar,120.0,2.0,500,1
Eat & Talk,250.0,3.0,500,1
Cathy Confectioners,300.0,3.0,500,1
THE ARTISAN HOUSE BAKERY AND CAFE,500.0,4.0,500,1
Moti Mahal Delux tandoori trail,800.0,3.0,500,1
Wat-a-Burger! India Ka Burger,250.0,4.0,500,1
